# Chapter 7 — LangGraph Basics II: Routers and Tool Nodes

Fully annotated Colab notebook covering Sections **7.1–7.9**.

Run this in Google Colab or VS Code. Each section mirrors the book content with
runnable Python examples.

## 7.1 Concept Flow — From Linear Graphs to Intelligent Routing

**Concept:** A *router node* looks at the user query and decides which specialist
(tool node) should handle it.

In this mini‑demo we will:

1. Define three tools: `calculator`, `search`, and `summarizer`.
2. Implement a simple router that inspects the query text.
3. Show a few example queries and which tool the router picks.

In [1]:
# 7.1 — Minimal router with three tools
import re
from dataclasses import dataclass
from typing import Callable, Dict

@dataclass
class Tool:
    name: str
    fn: Callable[[str], str]

def calculator_tool(query: str) -> str:
    """Tiny demo calculator: supports +, -, *, / on integers only.
    DO NOT use eval like this in production – this is purely for teaching.
    """
    expr = re.sub(r"[^0-9+\-*/()]", "", query)
    if not expr:
        return "Calculator: I couldn't find a math expression."
    try:
        result = eval(expr)
        return f"Calculator result for `{expr}` = {result}"
    except Exception as e:
        return f"Calculator error while evaluating `{expr}`: {e}"

def search_tool(query: str) -> str:
    """Stub search tool.
    In a real system this would call a web search API.
    """
    return f"[SearchTool] Would search the web for: '{query}'"

def summarizer_tool(query: str) -> str:
    """Stub summarizer.
    Real implementation would call an LLM.
    """
    return f"[Summarizer] Short, clear answer about: '{query}'"

tools: Dict[str, Tool] = {
    "calculator": Tool("calculator", calculator_tool),
    "search": Tool("search", search_tool),
    "summarizer": Tool("summarizer", summarizer_tool),
}

# Simple pattern helpers for the router
MATH_PATTERN = re.compile(r"[0-9]+\s*[-+*/]")
NEWS_KEYWORDS = ("today", "latest", "current", "news")

def simple_router_policy(query: str) -> str:
    """Return the name of the tool that should handle this query."""
    text = query.lower()
    if MATH_PATTERN.search(text):
        return "calculator"
    if any(kw in text for kw in NEWS_KEYWORDS):
        return "search"
    return "summarizer"

def run_router(query: str) -> str:
    tool_name = simple_router_policy(query)
    tool = tools[tool_name]
    result = tool.fn(query)
    return f"Router chose: {tool.name}\n{result}"

# Demo
examples = [
    "What is 12 + 7?",
    "Summarize today's top AI news.",
    "Explain LangGraph routers in one sentence.",
]
for q in examples:
    print("Query:", q)
    print(run_router(q))
    print("-" * 60)


Query: What is 12 + 7?
Router chose: calculator
Calculator result for `12+7` = 19
------------------------------------------------------------
Query: Summarize today's top AI news.
Router chose: search
[SearchTool] Would search the web for: 'Summarize today's top AI news.'
------------------------------------------------------------
Query: Explain LangGraph routers in one sentence.
Router chose: summarizer
[Summarizer] Short, clear answer about: 'Explain LangGraph routers in one sentence.'
------------------------------------------------------------


## 7.2 Math / CS Mini — Decision Policies in Routing

Formally, a router implements a **policy function**:

$$ r(x) = \operatorname*{argmax}_i p_i(x), $$

where:

- $x$ is the user query.
- $p_i(x)$ is the probability that tool $i$ is the correct handler.

In code, this is just: "compute scores for each tool, pick the index with
the largest score".


In [2]:
# 7.2 — Probability-based routing + reliability and cost
from typing import List, Tuple

tools_list = ["calculator", "search", "summarizer"]

def argmax(scores: List[float]) -> int:
    """Return index of maximum element.
    If there are ties, the first maximum is returned.
    """
    best_i = 0
    best_val = scores[0]
    for i, s in enumerate(scores[1:], start=1):
        if s > best_val:
            best_val = s
            best_i = i
    return best_i

def router_argmax_example():
    # Example raw probabilities p_i(x)
    p_calc, p_search, p_summ = 0.65, 0.25, 0.10
    probs = [p_calc, p_search, p_summ]
    i = argmax(probs)
    print("Raw probabilities:")
    for t, p in zip(tools_list, probs):
        print(f"  {t:11s}: {p:.2f}")
    print(f"\nargmax -> choose tool index {i} = '{tools_list[i]}'\n")

router_argmax_example()

# --- Reliability weighting ---
def reliability_weighting_example():
    # failure rates e_i and reliability w_i = 1 - e_i
    failure_rates = [0.02, 0.15, 0.05]  # calc, search, summarizer
    reliabilities = [1 - e for e in failure_rates]
    probs = [0.65, 0.25, 0.10]
    print("Reliabilities (w_i):")
    for t, w in zip(tools_list, reliabilities):
        print(f"  {t:11s}: w = {w:.2f}")
    # expected success if we follow raw router decision (here: calculator)
    expected_success = sum(p * w for p, w in zip(probs, reliabilities))
    print(f"\nExpected success E[S] = sum(p_i * w_i) = {expected_success:.3f}")

reliability_weighting_example()

# --- Latency & cost utility ---
def latency_cost_policy_example(alpha: float = 1.0, beta: float = 50.0):
    probs = [0.65, 0.25, 0.10]
    reliabilities = [0.98, 0.85, 0.92]
    latencies = [0.02, 0.42, 0.12]   # seconds per call
    costs = [0.0001, 0.002, 0.0008] # dollars per call

    eff_scores = []
    for w, t, c in zip(reliabilities, latencies, costs):
        eta = w / (alpha * t + beta * c)
        eff_scores.append(eta)

    decision_scores = [p * eta for p, eta in zip(probs, eff_scores)]

    print("Utility-weighted efficiency scores (eta_i):")
    for tname, eta in zip(tools_list, eff_scores):
        print(f"  {tname:11s}: eta = {eta:.2f}")

    print("\nFinal decision scores p_i * eta_i:")
    for tname, s in zip(tools_list, decision_scores):
        print(f"  {tname:11s}: score = {s:.2f}")

    i = argmax(decision_scores)
    print(f"\nRouter chooses (utility-weighted): {tools_list[i]}\n")

latency_cost_policy_example()


Raw probabilities:
  calculator : 0.65
  search     : 0.25
  summarizer : 0.10

argmax -> choose tool index 0 = 'calculator'

Reliabilities (w_i):
  calculator : w = 0.98
  search     : w = 0.85
  summarizer : w = 0.95

Expected success E[S] = sum(p_i * w_i) = 0.945
Utility-weighted efficiency scores (eta_i):
  calculator : eta = 39.20
  search     : eta = 1.63
  summarizer : eta = 5.75

Final decision scores p_i * eta_i:
  calculator : score = 25.48
  search     : score = 0.41
  summarizer : score = 0.58

Router chooses (utility-weighted): calculator



### 7.2 — Argmax for a School Kid (Satyam's Explanation)

Imagine three jars of candies:

- Jar A has **7 candies**.
- Jar B has **3 candies**.
- Jar C has **5 candies**.

If I ask: *"Which jar should I choose to get the most candy?"* you look at all
three numbers and pick the **largest one**.

Mathematically, we write:

$$ \arg\max\{7, 3, 5\} = 7, $$

but we usually return the **jar index**, e.g. `0` for A.


In [3]:
# Argmax demo for integers (candy jars)
candies = [7, 3, 5]
labels = ["Jar A", "Jar B", "Jar C"]

idx = argmax(candies)
print("Candies per jar:")
for lab, c in zip(labels, candies):
    print(f"  {lab}: {c}")
print(f"\nargmax -> choose index {idx} = {labels[idx]} (has {candies[idx]} candies)")


Candies per jar:
  Jar A: 7
  Jar B: 3
  Jar C: 5

argmax -> choose index 0 = Jar A (has 7 candies)


## 7.3 Satyam’s Explanation — Classroom Analogy

Think of a classroom with one teacher and three assistants:

- **Math assistant** → solves equations.
- **Research assistant** → looks up facts in books or on the web.
- **Writing assistant** → turns messy ideas into clean paragraphs.

When a student asks a question, the teacher (router) doesn’t answer directly.
Instead, the teacher first decides **which assistant should handle it**.

That is exactly what a router node does in LangGraph.


## 7.4 Workflow / Demos — Building a Multi‑Tool Router Graph

We now build a tiny **router graph** with:

1. A **router node** that inspects the query.
2. Three **tool nodes**: calculator, search, summarizer.
3. A **fallback node** used when the router is unsure.

We will also keep a small `trace` list to record which nodes were visited.

In [4]:
# 7.4 — Router graph execution with trace
from typing import Any, Dict, List

class RouterGraph:
    def __init__(self):
        self.trace: List[str] = []

    def route(self, query: str) -> str:
        self.trace.clear()
        self.trace.append(f"Router received: {query}")

        tool_name = simple_router_policy(query)
        # Simple confidence heuristic: if no obvious pattern, fall back
        if tool_name == "summarizer" and len(query.split()) < 3:
            self.trace.append("Low information query → fallback LLM")
            return self.fallback_node(query)

        self.trace.append(f"Router chose tool: {tool_name}")
        tool = tools[tool_name]
        result = tool.fn(query)
        self.trace.append(f"Tool '{tool_name}' finished successfully")
        return result

    def fallback_node(self, query: str) -> str:
        self.trace.append("Fallback node handling query")
        return f"[Fallback LLM] Safe generic answer for: '{query}'"

graph = RouterGraph()
answer = graph.route("48 * 17")
print("Answer:\n", answer, "\n", sep="")
print("Execution trace:")
for step in graph.trace:
    print(" •", step)


Answer:
Calculator result for `48*17` = 816

Execution trace:
 • Router received: 48 * 17
 • Router chose tool: calculator
 • Tool 'calculator' finished successfully


## 7.5 Comparison — Static Graphs vs Router Graphs

We contrast two designs:

1. **Static graph / chain**: always call tools in the same fixed order.
2. **Router graph**: choose tools dynamically.

We simulate latency to show the benefit of routing.

In [5]:
# 7.5 — Static chain vs router graph latency simulation
import time

def static_chain(query: str) -> str:
    """Always: summarizer → search → calculator (silly but illustrative)."""
    start = time.time()
    _ = summarizer_tool(query)
    _ = search_tool(query)
    result = calculator_tool(query)
    end = time.time()
    return result, end - start

def router_pipeline(query: str) -> str:
    start = time.time()
    result = run_router(query)
    end = time.time()
    return result, end - start

q_math = "What is 123 * 47?"
print("Static chain on math query:")
res_chain, t_chain = static_chain(q_math)
print(res_chain)
print(f"Time (approx): {t_chain*1000:.2f} ms\n")

print("Router pipeline on math query:")
res_router, t_router = router_pipeline(q_math)
print(res_router)
print(f"Time (approx): {t_router*1000:.2f} ms\n")


Static chain on math query:
Calculator result for `123*47` = 5781
Time (approx): 0.03 ms

Router pipeline on math query:
Router chose: calculator
Calculator result for `123*47` = 5781
Time (approx): 0.03 ms



## 7.6 Exercises (Notebook Version)

These mirror the book exercises. Treat them as coding prompts.

1. **Router Function Design**  
   Implement a rule‑based router function:
   - `T1` if the query contains numbers (math).
   - `T2` if the query contains keywords like `"search"`, `"find"`.
   - `T3` otherwise.

2. **Reliability Calculation**  
   Tools have reliabilities `w = [0.92, 0.88, 0.99]` and chosen with
   probabilities `p = [0.5, 0.3, 0.2]`. Compute
   `E[S] = sum(p_i * w_i)` in code.

3. **Routing Graph Sketch**  
   Extend `RouterGraph` to include a `CodeExec` tool and one fallback node.

Use extra code cells below to try your own solutions.

In [6]:
# 7.6 — Scratch space for your own solutions
pass


## 7.8 Industry Skill Tie‑In — Metrics for Multi‑Tool Routing

In production, teams track routing behavior as **metrics**:

- **Activation ratio**: how often each tool is selected.
- **Cost savings**: how many queries were handled by cheap tools instead of the LLM.
- **Error isolation**: failures that stayed local to one tool.
- **Coverage**: fraction of queries handled successfully.

We simulate a tiny routing log and compute these metrics.

In [7]:
# 7.8 — Simple routing metrics on synthetic logs
from collections import Counter

log = [
    {"query": "2+2", "tool": "calculator", "ok": True,  "cost": 0.0001},
    {"query": "today's AI news", "tool": "search", "ok": True,  "cost": 0.002},
    {"query": "summarize this", "tool": "summarizer", "ok": True,  "cost": 0.0008},
    {"query": "hard legal question", "tool": "llm", "ok": True,  "cost": 0.02},
    {"query": "broken search", "tool": "search", "ok": False, "cost": 0.002},
]

tool_counts = Counter(entry["tool"] for entry in log)
total = len(log)
print("Activation ratio (per tool):")
for tool, cnt in tool_counts.items():
    print(f"  {tool:11s}: {cnt}/{total} = {cnt/total:.2f}")

# Cost savings vs hypothetical 'always call LLM' policy
actual_cost = sum(e["cost"] for e in log)
all_llm_cost = total * 0.02  # assume 0.02$ per LLM call
savings = all_llm_cost - actual_cost
print(f"\nActual cost = ${actual_cost:.4f}")
print(f"All-LLM cost = ${all_llm_cost:.4f}")
print(f"Cost savings from routing = ${savings:.4f}")

# Coverage and error isolation
ok = sum(1 for e in log if e["ok"])
print(f"\nCoverage (successful queries) = {ok}/{total} = {ok/total:.2%}")


Activation ratio (per tool):
  calculator : 1/5 = 0.20
  search     : 2/5 = 0.40
  summarizer : 1/5 = 0.20
  llm        : 1/5 = 0.20

Actual cost = $0.0249
All-LLM cost = $0.1000
Cost savings from routing = $0.0751

Coverage (successful queries) = 4/5 = 80.00%


## 7.9 Closing Visual — Multi‑Tool Reasoning Stack (Code View)

Conceptual stack:

1. **LLM & External Systems** — base capabilities.
2. **Graph Execution Engine** — LangGraph runtime.
3. **Tool Nodes** — search, calculator, summarizer, etc.
4. **Router Policy Layer** — decides which tool to call.
5. **Monitoring & Routing Logs** — shows how the agent thinks.

We emulate this by running a query through our router graph and printing
a structured summary of what happened.

In [8]:
# 7.9 — Stack-style summary of one routed query
def explain_stack_flow(query: str):
    print("LLM & External Systems: available tools =", list(tools.keys()))
    print("Graph Execution Engine: executing RouterGraph.route(...)\n")
    result = graph.route(query)
    print("Router Policy Layer: decisions / trace:")
    for step in graph.trace:
        print("  ·", step)
    print("\nFinal answer from tool layer:\n", result)

explain_stack_flow("Summarize today's AI research news in one sentence")


LLM & External Systems: available tools = ['calculator', 'search', 'summarizer']
Graph Execution Engine: executing RouterGraph.route(...)

Router Policy Layer: decisions / trace:
  · Router received: Summarize today's AI research news in one sentence
  · Router chose tool: search
  · Tool 'search' finished successfully

Final answer from tool layer:
 [SearchTool] Would search the web for: 'Summarize today's AI research news in one sentence'
